<a href="https://colab.research.google.com/github/Dimildizio/bfs_pyg/blob/master/x_y_dict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
baseline_direction_vectors = {'vertical':   [np.array([0, -1]),
                                    np.array([0, 1])],

                              'horizontal': [np.array([-1, 0]),
                                             np.array([1, 0])],}


def find_all_lines(points, img, orientation='horizontal', n_removed_points=0, verbose=True,
                    save_images=True, output_dir='output_imgs/'):

    if n_removed_points > 0:
        print(f'Removing {n_removed_points} points')
    line_num_dict = {}    #<---------------------------------- Added to return
    chosen_indices = np.random.choice(np.arange(len(centers_array)),
                                        len(centers_array) - n_removed_points,
                                        replace=False)

    remaining_centers = points[chosen_indices]


    plt.figure(figsize=(20, 20))
    plt.imshow(img)

    i = 0
    result_lines = []
    while len(remaining_centers) != 0:
        i += 1

        print(f'{orientation} Line {i}')

        line_points = find_first_line(
            remaining_centers,
            img,
            direction_vectors=baseline_direction_vectors[orientation],
            verbose=verbose,
            save_images=save_images,
            output_dir=os.path.join(output_dir, f'line_{i}'),
        )

        line_points_set = {tuple(point) for point in line_points}
        remaining_centers = {tuple(point) for point in remaining_centers}

        assert len(remaining_centers) - len(line_points_set) == len(remaining_centers - line_points_set)

        result_lines.append(line_points)

        remaining_centers = remaining_centers - line_points_set
        remaining_centers = np.array(list(remaining_centers))
        for point in line_points:
            center = tuple(point)
            line_num_dict[center] = i   # <------------------------- added to return center:number

        plt.scatter(line_points[:, 0], line_points[:, 1])

        if orientation == 'horizontal':
            plt.text(*line_points[line_points[:, 0].argsort()][-1], f'{i}', c='white')
        elif orientation == 'vertical':
            plt.text(*line_points[line_points[:, 1].argsort()][0], f'{i}', c='white')

    return result_lines, line_num_dict # <--------------------------  return tuple

Execture the following

In [ ]:
horizontal_lines, points_dict = find_all_lines(centers_array, sample_image, orientation='horizontal', verbose=False, save_images=False);

In [ ]:
# Get lines\floors
sorted_points_dict = {}
for key, value in points_dict.items():
    if value not in sorted_points_dict:
        sorted_points_dict[value] = []

    sorted_points_dict[value].append(key)

for value in sorted_points_dict:
    sorted_points_dict[value] = sorted(sorted_points_dict[value], key=lambda point: point[0])

# Create the final dictionary with (center_x, center_y): (N, M) format
final_dict = {}
for value, points in sorted_points_dict.items():
    for i, point in enumerate(points, start=1):
        final_dict[point] = (value, i)


# Turn upside down
max_line = max(value[0] for value in final_dict.values())
for key, value in final_dict.items():
    N, M = value
    new_N = max_line - N + 1
    final_dict[key] = (new_N, M)


plt.figure(figsize=(20, 20))
plt.imshow(sample_image)
#Show colors and lines
line_colors = {i: (random.random(), random.random(), random.random()) for i in range(1, max_line + 1)}
for point, value in final_dict.items():
    x, y = point
    N, M = value
    color = line_colors[N]
    plt.scatter(x, y, c=color)
    plt.text(x, y, f'({N},{M})', color='white', fontsize=14, ha='center', va='center')

plt.show()
